## Solución Practica

#### Objetivo: Predecir el precio del airbnb a partir del dataset proporcionado.

In [3]:
import sys  
sys.path.insert(1, "C:/Users/ASUS/Documents/GitHub/Machine-Learning-101")

In [5]:
from utils import plot_decision_boundary, poly_linear_regression, CM_BRIGHT

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

## 1. Carga de datos y división train/test

In [293]:
from sklearn.model_selection import train_test_split

airbnb_data = pd.read_csv("./airbnb-listings-extract.csv", delimiter=";")
train, test = train_test_split(airbnb_data, test_size=0.3, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardar datos por separado
train.to_csv('./airbnb-listings_train.csv', sep=';', decimal='.', index=False)
test.to_csv('./airbnb-listings_test.csv', sep=';', decimal='.', index=False)

# Cargar el dataset de train
airbnb_train_data = pd.read_csv('./airbnb-listings_train.csv', sep=';', decimal='.')

Dimensiones del dataset de training: (10346, 89)
Dimensiones del dataset de test: (4434, 89)


#### **** Cargar el dataset de train ****

In [200]:
# Cargar el dataset de train
airbnb_train_data = pd.read_csv('./airbnb-listings_train.csv', sep=';', decimal='.')

airbnb_train_data.shape

(10346, 89)

In [186]:
# Visualizar 5 primeras filas
airbnb_train_data.head(5).T

,0,1,2,3,4
ID,2156319,3377153,14800635,11185674,18092171
Listing Url,https://www.airbnb.com/rooms/2156319,https://www.airbnb.com/rooms/3377153,https://www.airbnb.com/rooms/14800635,https://www.airbnb.com/rooms/11185674,https://www.airbnb.com/rooms/18092171
Scrape ID,20170407214119,20170407214119,20170407214119,20170407214119,20170407214119
Last Scraped,2017-04-08,2017-04-08,2017-04-08,2017-04-08,2017-04-08
Name,GRANT VII Plaza Mayor,Nice flat in Plaza Mayor (lift),* ROOM double Barrio Salamanca *,cheap apartment in Madrid city cent,"ACOGEDOR APARTAMENTO EN MADRID, CALLE MANZANARES"
Summary,NaN,"Flat is in the centre town, really close to Pl...","Private room for two persons has a double bed,...","Cheap apartment, next to Madrid rio, just four...","Acogedor apartamento, para parejas y/o familia."
Space,Beautiful and charming apartment recently deco...,You cannot find a better location to stay in M...,It is a penthouse located in the salamanca dis...,NaN,NaN
Description,Beautiful and charming apartment recently deco...,"Flat is in the centre town, really close to Pl...","Private room for two persons has a double bed,...","Cheap apartment, next to Madrid rio, just four...","Acogedor apartamento, para parejas y/o familia."
Experiences Offered,none,none,none,none,none
Neighborhood Overview,NaN,"It´s really cool, with a lot of new business (...",The District of Salamanca is one of the 21 dis...,NaN,NaN


## 2. Análisis exploratorio

### Analizar la estructura, el contenido del dataset e identifica la columna "precio" que se usará como variable objetivo.

In [188]:
airbnb_train_data.columns

Index(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview',
       'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
       'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location',
       'Host About', 'Host Response Time', 'Host Response Rate',
       'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url',
       'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price', 'Weekly Price',
       'Month

In [145]:
pd.set_option('display.max_columns', None) # Mostrar todas las columnas
pd.set_option('display.float_format', '{:,.6f}'.format) # Mostrar los números sin notación científica
airbnb_train_data.describe(include="all")

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,Notes,Transit,Access,Interaction,House Rules,Thumbnail Url,Medium Url,Picture Url,XL Picture Url,Host ID,Host URL,Host Name,Host Since,Host Location,Host About,Host Response Time,Host Response Rate,Host Acceptance Rate,Host Thumbnail Url,Host Picture Url,Host Neighbourhood,Host Listings Count,Host Total Listings Count,Host Verifications,Street,Neighbourhood,Neighbourhood Cleansed,Neighbourhood Group Cleansed,City,State,Zipcode,Market,Smart Location,Country Code,Country,Latitude,Longitude,Property Type,Room Type,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Square Feet,Price,Weekly Price,Monthly Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Calendar Updated,Has Availability,Availability 30,Availability 60,Availability 90,Availability 365,Calendar last Scraped,Number of Reviews,First Review,Last Review,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
count,"10,346.000000",10346,"10,346.000000",10346,10345,9922,7619,10340,10346,6397,3960,6327,5814,5747,6738,8319,8319,10329,8319,"10,346.000000",10346,10344,10344,10318,6692,9016,"9,016.000000",25,10344,10344,7665,"10,344.000000","10,344.000000",10340,10346,6706,10346,9645,10342,10240,9999,10307,10346,10346,10345,"10,346.000000","10,346.000000",10346,10346,"10,346.000000","10,308.000000","10,326.000000","10,312.000000",10346,10215,410.000000,"10,332.000000","2,513.000000","2,502.000000","4,447.000000","6,124.000000","10,346.000000","10,346.000000","10,346.000000","10,346.000000",10346,7,"10,346.000000","10,346.000000","10,346.000000","10,346.000000",10346,"10,346.000000",8100,8099,"7,995.000000","7,976.000000","7,980.000000","7,971.000000","7,980.000000","7,969.000000","7,967.000000",251,143,10346,"10,342.000000","8,100.000000",10346,10345
unique,NaN,10346,NaN,35,10112,9358,7203,10029,5,5673,3361,5599,5083,4736,5745,8317,8317,10326,8317,NaN,6706,2713,1990,468,3822,4,NaN,9,6681,6681,332,NaN,NaN,178,1328,325,421,45,209,89,498,41,217,18,17,NaN,NaN,21,3,NaN,NaN,NaN,NaN,5,8914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,1,NaN,NaN,NaN,NaN,37,NaN,1531,698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230,11,8,NaN,NaN,10346,82
top,NaN,https://www.airbnb.com/rooms/2156319,NaN,2017-04-08,Apartamento en el centro de Madrid,"Unique apartment in vibrant neighborhoods, car...",Los Apartamentos Good Stay Prado se encuentran...,Es un piso con 6 habitaciones de las que 5 ha...,none,Se trata de una de las zonas más emblemáticas ...,"Check in time is at 3pm or later, although som...","It is calm, quite zone and you don’t have to s...","Wifi, TV, washing machine, Microwave, full equ...",We will arrange the check in time depending on...,Checkin is done at our office. The address: Ca...,https://a0.muscache.com/im/pictures/4fb56b8a-2...,https://a0.muscache.com/im/pictures/4fb56b8a-2...,https://public.opendatasoft.com/api/v2/catalog...,https://a0.muscache.com/im/pictures/4fb56b8a-2...,NaN,https://www.airbnb.com/users/show/54310958,Javier,2016-01-12,"Madrid, Community of Madrid, Spain",Be Mate offers the exclusive City Mate service...,within an hour,NaN,100%,https://a0.muscache.com/im/pictures/8f3ec0f2-1...,https://a0.muscache.com/im/pictures/8f3ec0f2-1...,Malasaña,NaN,NaN,"email,phone,reviews,jumio","Embajadores, Madrid, Comunidad de Madrid 28012...",Malasaña,Embajadores,Centro,Madrid,Comunidad de Madrid,28012,Madrid,"Madrid, Spain",ES,Spain,NaN,NaN,Apartment,Entire home/apt,NaN,NaN,NaN,NaN,Real Bed,"TV,Internet,Wireless Internet,Air conditioning...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,today,t,NaN,NaN,NaN,NaN,2017-04-08,NaN,2017-02-26,2017-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,City registration pending,Paris,strict,NaN,NaN,"40.415

### **** Eliminando columnas que considero que no son relevantes para el análisis. **** 

In [202]:
def eliminar_columnas (dataset): 
    return dataset.drop(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Description', 'Experiences Offered', 'Neighborhood Overview',
       'Notes', 'Transit', 'Interaction', 'House Rules',
       'Host ID', 'Host Name', 'Host Location', 'Picture Url', 'Host URL',
       'Host Thumbnail Url', 'XL Picture Url', 'Thumbnail Url', 'Medium Url', 'Host Picture Url', 'First Review', 'Last Review'], axis=1)

print(f'Cantidad de columnas antes de borrar: {airbnb_train_data.shape[1]}')
airbnb_train_data = eliminar_columnas(airbnb_train_data)
print(f'Cantidad de columnas después de borrar: {airbnb_train_data.shape[1]}')

Cantidad de columnas antes de borrar: 89
Cantidad de columnas después de borrar: 66


### Analizar campos con valores Null y decidir que hacer con ellos.

In [204]:
pd.set_option('display.max_columns', None) # Mostrar todas las columnas
pd.set_option('display.float_format', '{:,.6f}'.format) # Mostrar los números sin notación científica
airbnb_train_data.describe(include="all")

,Space,Access,Host Since,Host About,Host Response Time,Host Response Rate,Host Acceptance Rate,Host Neighbourhood,Host Listings Count,Host Total Listings Count,Host Verifications,Street,Neighbourhood,Neighbourhood Cleansed,Neighbourhood Group Cleansed,City,State,Zipcode,Market,Smart Location,Country Code,Country,Latitude,Longitude,Property Type,Room Type,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Square Feet,Price,Weekly Price,Monthly Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Calendar Updated,Has Availability,Availability 30,Availability 60,Availability 90,Availability 365,Calendar last Scraped,Number of Reviews,First Review,Last Review,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
count,7619,5814,10344,6692,9016,"9,016.000000",25,7665,"10,344.000000","10,344.000000",10340,10346,6706,10346,9645,10342,10240,9999,10307,10346,10346,10345,"10,346.000000","10,346.000000",10346,10346,"10,346.000000","10,308.000000","10,326.000000","10,312.000000",10346,10215,410.000000,"10,332.000000","2,513.000000","2,502.000000","4,447.000000","6,124.000000","10,346.000000","10,346.000000","10,346.000000","10,346.000000",10346,7,"10,346.000000","10,346.000000","10,346.000000","10,346.000000",10346,"10,346.000000",8100,8099,"7,995.000000","7,976.000000","7,980.000000","7,971.000000","7,980.000000","7,969.000000","7,967.000000",251,143,10346,"10,342.000000","8,100.000000",10346,10345
unique,7203,5083,1990,3822,4,NaN,9,332,NaN,NaN,178,1328,325,421,45,209,89,498,41,217,18,17,NaN,NaN,21,3,NaN,NaN,NaN,NaN,5,8914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,1,NaN,NaN,NaN,NaN,37,NaN,1531,698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230,11,8,NaN,NaN,10346,82
top,Los Apartamentos Good Stay Prado se encuentran...,"Wifi, TV, washing machine, Microwave, full equ...",2016-01-12,Be Mate offers the exclusive City Mate service...,within an hour,NaN,100%,Malasaña,NaN,NaN,"email,phone,reviews,jumio","Embajadores, Madrid, Comunidad de Madrid 28012...",Malasaña,Embajadores,Centro,Madrid,Comunidad de Madrid,28012,Madrid,"Madrid, Spain",ES,Spain,NaN,NaN,Apartment,Entire home/apt,NaN,NaN,NaN,NaN,Real Bed,"TV,Internet,Wireless Internet,Air conditioning...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,today,t,NaN,NaN,NaN,NaN,2017-04-08,NaN,2017-02-26,2017-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,City registration pending,Paris,strict,NaN,NaN,"40.4154180336,-3.70712273935","Host Has Profile Pic,Host Identity Verified,Is..."
freq,18,42,135,124,5544,NaN,14,687,NaN,NaN,2598,416,608,1290,4767,9249,7440,1441,9300,9243,9817,9817,NaN,NaN,8422,6330,NaN,NaN,NaN,NaN,10126,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2717,7,NaN,NaN,NaN,NaN,6967,NaN,43,539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,70,4062,NaN,NaN,1,1779
mean,NaN,NaN,NaN,NaN,NaN,94.846273,NaN,NaN,12.876450,12.876450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.485115,-3.708535,NaN,NaN,3.290064,1.289726,1.347376,2.053627,NaN,NaN,395.139024,74.242935,380.284123,"1,441.808153",188.551158,32.268452,1.586797,7.641601,3.148560,979.637444,NaN,NaN,8.979702,22.999130,39.838972,201.677170,NaN,22.453412,NaN,NaN,91.607380,9.411234,9.319799,9.619621,9.643609,9.535073,9.208736,NaN,NaN,NaN,9.658963,1.871260,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,15.124996,NaN,NaN,34.990589,34.990589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.673811,13.720947,NaN,NaN,2.096108,0.668635,0.904172,1.626308,NaN,NaN,667.520462,72.418780,195.799471,"1,265.861086",115.648243,31.545209,1.163591,11.662596,14.211265,"10,033.724779",NaN,NaN,9.358143,19.805690,29.735404,128.242731,NaN,37.736055,NaN,NaN,9.121333,0.932933,1.003058,0.810896,0.769159,0.776672,0.960288,NaN,NaN,NaN,23.879225,1.873803,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [206]:
def mostrar_valores_unicos_por_columna(dataset):
    print(f'Total Registros: {dataset.shape[0]} \n')
    
    for column in dataset.columns: 
        print(f'{column}: {airbnb_train_data[column].nunique()}')

mostrar_valores_unicos_por_columna(airbnb_train_data)

Total Registros: 10346 

Space: 7203
Access: 5083
Host Since: 1990
Host About: 3822
Host Response Time: 4
Host Response Rate: 65
Host Acceptance Rate: 9
Host Neighbourhood: 332
Host Listings Count: 54
Host Total Listings Count: 54
Host Verifications: 178
Street: 1328
Neighbourhood: 325
Neighbourhood Cleansed: 421
Neighbourhood Group Cleansed: 45
City: 209
State: 89
Zipcode: 498
Market: 41
Smart Location: 217
Country Code: 18
Country: 17
Latitude: 10346
Longitude: 10346
Property Type: 21
Room Type: 3
Accommodates: 16
Bathrooms: 17
Bedrooms: 10
Beds: 16
Bed Type: 5
Amenities: 8914
Square Feet: 82
Price: 335
Weekly Price: 340
Monthly Price: 480
Security Deposit: 111
Cleaning Fee: 101
Guests Included: 15
Extra People: 59
Minimum Nights: 41
Maximum Nights: 145
Calendar Updated: 56
Has Availability: 1
Availability 30: 31
Availability 60: 61
Availability 90: 91
Availability 365: 366
Calendar last Scraped: 37
Number of Reviews: 243
First Review: 1531
Last Review: 698
Review Scores Rating: 51
R

In [240]:
airbnb_train_data.dtypes

Host Since                       object
Host Response Time               object
Host Response Rate              float64
Host Acceptance Rate             object
Host Neighbourhood               object
Host Total Listings Count       float64
Host Verifications               object
Street                           object
Neighbourhood                    object
Neighbourhood Cleansed           object
Neighbourhood Group Cleansed     object
City                             object
State                            object
Zipcode                          object
Market                           object
Smart Location                   object
Country Code                     object
Country                          object
Latitude                        float64
Longitude                       float64
Property Type                    object
Room Type                        object
Accommodates                      int64
Bathrooms                       float64
Bedrooms                        float64


In [309]:
airbnb_train_data['Host Acceptance Rate'].unique()

array([nan, '100%', '67%', '0%', '85%', '96%', '74%', '95%', '88%', '80%'],
      dtype=object)

In [208]:
airbnb_train_data['Host Listings Count'].unique()

array([ 40.,   1.,   3.,   4.,   2.,   8.,  15.,  10.,   5.,  57.,  30.,
         9.,  68.,   6.,  23.,  25.,   7.,  16.,  38., 147., 142.,  17.,
        20.,  13., 164., 207.,  14.,  46.,  11.,  74.,  12.,  28.,  24.,
        34., 114.,  21., 136.,  32.,  27., 162.,  39.,  18.,  31.,  26.,
        51.,  45.,   0.,  36.,  84.,  35.,  22.,  nan,  81., 265., 519.])

In [210]:
airbnb_train_data['Host Total Listings Count'].unique()

array([ 40.,   1.,   3.,   4.,   2.,   8.,  15.,  10.,   5.,  57.,  30.,
         9.,  68.,   6.,  23.,  25.,   7.,  16.,  38., 147., 142.,  17.,
        20.,  13., 164., 207.,  14.,  46.,  11.,  74.,  12.,  28.,  24.,
        34., 114.,  21., 136.,  32.,  27., 162.,  39.,  18.,  31.,  26.,
        51.,  45.,   0.,  36.,  84.,  35.,  22.,  nan,  81., 265., 519.])

In [232]:
airbnb_train_data['Host Acceptance Rate'].unique()

array([nan, '100%', '67%', '0%', '85%', '96%', '74%', '95%', '88%', '80%'],
      dtype=object)

In [252]:
airbnb_train_data['Features'].unique()

array(['Host Has Profile Pic,Host Identity Verified,Is Location Exact,Requires License',
       'Host Has Profile Pic,Host Identity Verified,Is Location Exact,Requires License,Instant Bookable',
       'Host Has Profile Pic,Host Identity Verified,Requires License,Instant Bookable',
       'Host Has Profile Pic,Is Location Exact,Requires License',
       'Host Has Profile Pic,Requires License',
       'Host Has Profile Pic,Host Identity Verified,Is Location Exact,Instant Bookable',
       'Host Has Profile Pic,Is Location Exact,Requires License,Instant Bookable',
       'Host Has Profile Pic,Host Identity Verified,Is Location Exact,Requires License,Require Guest Profile Picture,Require Guest Phone Verification',
       'Host Is Superhost,Host Has Profile Pic,Host Identity Verified,Is Location Exact,Requires License',
       'Host Has Profile Pic,Host Identity Verified,Requires License',
       'Host Has Profile Pic,Requires License,Instant Bookable',
       'Host Has Profile Pic,Host Id

In [276]:
pd.set_option('display.max_columns', None) # Mostrar todas las columnas
pd.set_option('display.float_format', '{:,.6f}'.format) # Mostrar los números sin notación científica
airbnb_train_data.describe(include="O")

,Host Since,Host Response Time,Host Acceptance Rate,Host Neighbourhood,Host Verifications,Street,Neighbourhood,Neighbourhood Cleansed,Neighbourhood Group Cleansed,City,State,Zipcode,Market,Smart Location,Country Code,Country,Property Type,Room Type,Bed Type,Amenities,Has Availability,First Review,Last Review,License,Jurisdiction Names,Cancellation Policy,Geolocation,Features
count,10344,9016,25,7665,10340,10346,6706,10346,9645,10342,10240,9999,10307,10346,10346,10345,10346,10346,10346,10215,7,8100,8099,251,143,10346,10346,10345
unique,1990,4,9,332,178,1328,325,421,45,209,89,498,41,217,18,17,21,3,5,8914,1,1531,698,230,11,8,10346,82
top,2016-01-12,within an hour,100%,Malasaña,"email,phone,reviews,jumio","Embajadores, Madrid, Comunidad de Madrid 28012...",Malasaña,Embajadores,Centro,Madrid,Comunidad de Madrid,28012,Madrid,"Madrid, Spain",ES,Spain,Apartment,Entire home/apt,Real Bed,"TV,Internet,Wireless Internet,Air conditioning...",t,2017-02-26,2017-04-02,City registration pending,Paris,strict,"40.4154180336,-3.70712273935","Host Has Profile Pic,Host Identity Verified,Is..."
freq,135,5544,14,687,2598,416,608,1290,4767,9249,7440,1441,9300,9243,9817,9817,8422,6330,10126,33,7,43,539,9,70,4062,1,1779


#### **** Eliminar columnas redundantes y no relevantes **** 

In [228]:
def eliminar_columnas_redundantes (dataset): 
    return dataset.drop(['Host Listings Count', 'Access', 'Space', 'Host About', 'Weekly Price', 'Monthly Price', 'Calendar Updated', 'Calendar last Scraped', 'Calculated host listings count', 'Features'], axis=1)

print(f'Cantidad de columnas antes de borrar: {airbnb_train_data.shape[1]}')
airbnb_train_data = eliminar_columnas_redundantes(airbnb_train_data)
print(f'Cantidad de columnas después de borrar: {airbnb_train_data.shape[1]}')

Cantidad de columnas antes de borrar: 66
Cantidad de columnas después de borrar: 57


In [260]:
pd.set_option('display.max_rows', None)
airbnb_train_data.isnull().sum()

Host Since                          2
Host Response Time               1330
Host Response Rate               1330
Host Acceptance Rate            10321
Host Neighbourhood               2681
Host Total Listings Count           2
Host Verifications                  6
Street                              0
Neighbourhood                    3640
Neighbourhood Cleansed              0
Neighbourhood Group Cleansed      701
City                                4
State                             106
Zipcode                           347
Market                             39
Smart Location                      0
Country Code                        0
Country                             1
Latitude                            0
Longitude                           0
Property Type                       0
Room Type                           0
Accommodates                        0
Bathrooms                          38
Bedrooms                           20
Beds                               34
Bed Type    

In [262]:
airbnb_train_data[['ID', 'Price']]

KeyError: "['ID'] not in index"

In [96]:
airbnb_train_data[['ID', 'Price', 'Weekly Price', 'Monthly Price']].isnull().sum()

ID                  0
Price              14
Weekly Price     7833
Monthly Price    7844
dtype: int64

In [58]:
airbnb_train_data.isnull().sum()

ID                                   0
Listing Url                          0
Scrape ID                            0
Last Scraped                         0
Name                                 1
                                  ... 
Cancellation Policy                  0
Calculated host listings count       4
Reviews per Month                 2246
Geolocation                          0
Features                             1
Length: 89, dtype: int64

In [44]:
airbnb_train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,10346.0,1.028354e+07,5.555731e+06,1.986400e+04,5.597253e+06,1.130555e+07,1.532606e+07,1.858361e+07
Scrape ID,10346.0,2.017038e+13,5.374658e+08,2.016010e+13,2.017041e+13,2.017041e+13,2.017041e+13,2.017062e+13
Host ID,10346.0,3.606270e+07,3.417818e+07,1.745300e+04,6.884224e+06,2.483964e+07,5.431096e+07,1.247534e+08
Host Response Rate,9016.0,9.484627e+01,1.512500e+01,0.000000e+00,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02
Host Listings Count,10344.0,1.287645e+01,3.499059e+01,0.000000e+00,1.000000e+00,2.000000e+00,6.000000e+00,5.190000e+02
Host Total Listings Count,10344.0,1.287645e+01,3.499059e+01,0.000000e+00,1.000000e+00,2.000000e+00,6.000000e+00,5.190000e+02
Latitude,10346.0,4.048512e+01,4.673811e+00,-3.785118e+01,4.040978e+01,4.041934e+01,4.043079e+01,5.594576e+01
Longitude,10346.0,-3.708535e+00,1.372095e+01,-1.224374e+02,-3.707615e+00,-3.700787e+00,-3.684274e+00,1.533714e+02
Accommodates,10346.0,3.290064e+00,2.096108e+00,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,1.600000e+01
Bathrooms,10308.0,1.289726e+00,6.686352e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,8.000000e+00
